<a href="https://colab.research.google.com/github/YellaCRE/SportRecog/blob/main/%EB%AC%B8%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91%2C%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyreadstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#데이터 불러오기

In [ ]:
import os
import re
import pandas as pd
pd.set_option('mode.chained_assignment',  None) #수정오류 무시
import numpy as np

In [ ]:
path = "/content/drive/MyDrive/2022 휴학/문화관광공모전/data/"
#csv 형식으로 된 데이터 파일을 읽어옵니다. 
data = pd.read_spss(path + "국민여가활동조사_통합데이터_2014-2021.sav")

연도

In [ ]:
#최근 5년이내 데이터만 수집
data = data[data['year'].isin([2021,2020,2019,2018])]

#Data

In [ ]:
df_ID = data[['ID','year']]
df_ID.head()

,ID,year
20636,1.0,2018.0
20637,2.0,2018.0
20638,3.0,2018.0
20639,4.0,2018.0
20640,5.0,2018.0


인구데이터

In [ ]:
df_per = data[['DM1','DM2','DM11','Q45','Q47_N','DM5']]
df_per.head()

,DM1,DM2,DM11,Q45,Q47_N,DM5
20636,남성,20대,서울,하지 않았다,NaN,미혼
20637,여성,50대,서울,하였다,56.0,기혼
20638,남성,40대,서울,하였다,48.0,기혼
20639,여성,30대,서울,하지 않았다,NaN,기혼
20640,여성,60대,서울,하지 않았다,NaN,기혼


Q1 경험한 여가활동

In [ ]:
col_list = data.columns
df_exp = data[col_list[col_list.str.contains('Q1_')==True]]
df_exp.replace('미참여',0,inplace=True)
df_exp.replace('참여',1,inplace=True)
df_exp.head()

,Q1_1,Q1_2,Q1_3,Q1_4,Q1_5,Q1_6,Q1_7,Q1_8,Q1_9,Q1_10,...,Q1_82,Q1_83,Q1_84,Q1_85,Q1_86,Q1_87,Q1_88,Q1_89,Q1_90,Q1_91
20636,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,1
20637,1,0,0,0,1,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,1
20638,0,0,0,0,1,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,1
20639,0,0,0,0,0,0,1,1,0,0,...,0,1,0,1,0,0,0,0,0,1
20640,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1


여가활동의 목적

In [ ]:
df_etc = data[['Q3']]
df_etc.head()

,Q3
20636,개인의 즐거움을 위해
20637,마음의 안정과 휴식을 위해
20638,마음의 안정과 휴식을 위해
20639,자기만족을 위해
20640,건강을 위해


가장 만족한 여가활동

In [ ]:
df_best = data[['Q4_1','Q4_2','Q4_3']]
df_best.head()

,Q4_1,Q4_2,Q4_3
20636,어학/기술/자격증 취득 공부/학원 등 이용,"친구만남/동호회 모임(2016, 2018)","볼링, 탁구"
20637,연극공연 관람(뮤지컬 포함),해외여행,"전시회 관람(미술, 사진, 건축, 디자인 등)"
20638,요리하기/다도,쇼핑/외식,"미용(피부관리, 헤어관리, 네일아트, 마사지, 성형 등)"
20639,해외여행,쇼핑/외식,요가/필라테스/태보
20640,수영,해외여행,"춤/무용(발레, 한국무용, 현대무용, 방송댄스, 스트릿댄스, 비보잉 등)"


1년간 가장 많이 한 여가활동

In [ ]:
df_past = data[['Q2_1_1','Q2_6_1','Q2_1_2','Q2_6_2','Q2_1_3','Q2_6_3','Q2_1_4','Q2_6_4','Q2_1_5','Q2_6_5']]
df_past.head()

,Q2_1_1,Q2_6_1,Q2_1_2,Q2_6_2,Q2_1_3,Q2_6_3,Q2_1_4,Q2_6_4,Q2_1_5,Q2_6_5
20636,인터넷 검색/1인 미디어 제작/SNS,5,게임(온라인/모바일/콘솔게임 등),5,"친구만남/동호회 모임(2016, 2018)",6,어학/기술/자격증 취득 공부/학원 등 이용,5,당구/포켓볼,6
20637,반려동물 돌보기,5,산책 및 걷기,4 보통이다,쇼핑/외식,5,TV시청(IPTV 포함),4 보통이다,영화관람,5
20638,TV시청(IPTV 포함),5,요리하기/다도,6,산책 및 걷기,5,쇼핑/외식,5,연극공연 관람(뮤지컬 포함),6
20639,인터넷 검색/1인 미디어 제작/SNS,7 매우 만족한다,요가/필라테스/태보,5,쇼핑/외식,5,"친구만남/동호회 모임(2016, 2018)",6,영화관람,6
20640,수영,2,산책 및 걷기,7 매우 만족한다,TV시청(IPTV 포함),6,"미용(피부관리, 헤어관리, 네일아트, 마사지, 성형 등)",6,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",5


평일,주말 스포츠활동

In [ ]:
df_week = data[['Q11_1_C','Q11_2_C','Q11_1_D','Q11_2_D','Q11_3_C','Q11_3_D']]
df_week.head()

,Q11_1_C,Q11_2_C,Q11_1_D,Q11_2_D,Q11_3_C,Q11_3_D
20636,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",5.0,당구/포켓볼,4.0,없음,"볼링, 탁구"
20637,NaN,NaN,헬스(보디빌딩)/에어로빅,4.0,없음,없음
20638,NaN,NaN,NaN,NaN,없음,없음
20639,NaN,NaN,요가/필라테스/태보,5.0,없음,요가/필라테스/태보
20640,"스포츠 경기 간접관람- TV, DMB를 통한관람(축구, 야구, 농구, 배구 등)",5.0,수영,5.0,없음,수영


In [ ]:
df_wknd = data[['Q12_1_C','Q12_2_C','Q12_1_D','Q12_2_D','Q12_3_C','Q12_3_D']]
df_wknd.head()

,Q12_1_C,Q12_2_C,Q12_1_D,Q12_2_D,Q12_3_C,Q12_3_D
20636,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",5.0,"농구, 배구, 야구, 축구, 족구",5.0,없음,없음
20637,NaN,NaN,없음,NaN,없음,없음
20638,NaN,NaN,NaN,NaN,없음,없음
20639,NaN,NaN,골프,6.0,없음,골프
20640,"스포츠 경기 직접관람- 경기장방문관람(축구, 야구,농구, 배구 등)",5.0,"볼링, 탁구",6.0,없음,"윈드서핑, 수상스키, 스킨스쿠버다이빙, 래프팅, 요트"


데이터 생성

In [ ]:
df = pd.concat([df_ID,df_per,df_etc,df_best,df_past,df_week,df_wknd,df_exp],axis=1)
df

,ID,year,DM1,DM2,DM11,Q45,Q47_N,DM5,Q3,Q4_1,...,Q1_82,Q1_83,Q1_84,Q1_85,Q1_86,Q1_87,Q1_88,Q1_89,Q1_90,Q1_91
20636,1.0,2018.0,남성,20대,서울,하지 않았다,NaN,미혼,개인의 즐거움을 위해,어학/기술/자격증 취득 공부/학원 등 이용,...,0,0,1,0,0,0,0,0,1,1
20637,2.0,2018.0,여성,50대,서울,하였다,56.0,기혼,마음의 안정과 휴식을 위해,연극공연 관람(뮤지컬 포함),...,0,0,1,1,0,0,0,0,0,1
20638,3.0,2018.0,남성,40대,서울,하였다,48.0,기혼,마음의 안정과 휴식을 위해,요리하기/다도,...,0,0,1,1,0,0,0,0,0,1
20639,4.0,2018.0,여성,30대,서울,하지 않았다,NaN,기혼,자기만족을 위해,해외여행,...,0,1,0,1,0,0,0,0,0,1
20640,5.0,2018.0,여성,60대,서울,하지 않았다,NaN,기혼,건강을 위해,수영,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61326,25065.0,2021.0,여성,40대,전남,하였다,30.0,기혼,스트레스 해소를 위해,지역축제 참가,...,0,1,1,0,1,0,0,0,0,0
61327,25066.0,2021.0,남성,60대,전남,하였다,50.0,기혼,자기만족을 위해,골프,...,0,1,0,1,1,0,0,0,0,0
61328,25067.0,2021.0,남성,15-19세,전남,하지 않았다,NaN,미혼,스트레스 해소를 위해,"배드민턴/줄넘기/맨손, 스트레칭 체조/훌라후프",...,0,1,0,0,1,0,0,0,0,0
61329,25068.0,2021.0,여성,60대,전남,하지 않았다,NaN,기혼,자기만족을 위해,자동차 드라이브,...,0,1,0,0,1,0,0,0,0,0


#person_data 전처리

근무시간

학생,주부 근무시간 입력

In [ ]:
temp_list = []
temp = data[['Q45','Q47_N']]

for n,d in enumerate(temp['Q45']):
  if d == '하였다':
    temp_list.append(temp['Q47_N'].iloc[n])
  else:
    temp_list.append(30) # 학생,주부 6시간5일근무 기준

df['Q47_N'] = temp_list

나이대,성별

In [ ]:
df['DM2'] = df['DM2'].str.replace(pat=r'[ㄱ-ㅣ가-힣]+', repl= r'', regex=True)
df['DM2'] = df['DM2'].str.replace('15-19','10')
df['DM1'] = df['DM1'].replace({'남성':'M', '여성':'F'})

시간대

In [ ]:
df = df.astype({'year':'string'})
df['year'] = df['year'].replace({'2018.0':'2018', '2019.0':'2019','2020.0':'2020','2021.0':'2021'})

일하는 시간

In [ ]:
df['Q47_N'] = df['Q47_N'].fillna(0)

여가활동의 목적

In [ ]:
#카디널리티 감소를 통한 성능향상을 목적으로 4가지 카테고리로 변경
df['Q3'] = df['Q3'].replace(['개인의 즐거움을 위해','스트레스 해소를 위해'],'즐거움')
df['Q3'] = df['Q3'].replace(['마음의 안정과 휴식을 위해','시간을 보내기 위해'],'휴식')
df['Q3'] = df['Q3'].replace(['자기만족을 위해','건강을 위해','자기 계발을 위해'],'자기계발')
df['Q3'] = df['Q3'].replace(['가족과 시간을 함께하기 위해','대인 관계·교제를 위해'],'대인관계')

여가활동 분류정리

In [ ]:
#여가활동 컬럼이름 csv
Q1c = pd.read_csv(path+'Q1_colname.csv')
#컬럼이름 리스트로 변경
temp = Q1c[Q1c['대분류']=='취미생활']
temp.대분류 = temp.대분류 + temp.중분류
temp2 = Q1c[Q1c['대분류']!='취미생활']
Q1c = pd.concat([temp,temp2]).sort_values('index',ascending=True).drop('중분류',axis=1)
col_list = Q1c.컬럼명.to_list()
ch_list = Q1c.대분류.to_list()
#여가활동 딕셔너리 생성
#취미생활남, 취미생활여 등등 카디널리티 감소
temp_dict = dict(zip(col_list,ch_list))
temp_dict['이성교제(데이트)/미팅/소개팅(2016, 2018)'] = '사교활동'
temp_dict['친구만남/동호회 모임(2016, 2018)'] = '사교활동'
temp_dict['독서/만화책(웹툰)보기(2016, 2018)'] = '취미생활중'
temp_dict['이성교제(데이트)/미팅/소개팅(2016, 2018)'] = '사교활동'
#전처리 적용
df=df.replace(temp_dict)

ID 전처리

In [ ]:
df=df.astype({'ID':'int'})
df=df.astype({'ID':'string'})
df=df.astype({'year':'string'})
#연도와 ID통합, ID에 distinct부여
df.ID = df.year + df.ID

데이터 저장

In [ ]:
df.to_csv(path+'person_data.csv')